Computing probabilities using Python

In [4]:
# Creating a sample space of coin flips
sample_space = {'Heads', 'Tails'}

# Computing the probability of heads
probability_heads = 1 / len(sample_space)

# Defining event conditions
def is_heads_or_tails(outcome): return outcome in {'Heads', 'Tails'}
def is_neither(outcome): return outcome not in {'Heads', 'Tails'}
def is_heads(outcome): return outcome == 'Heads'
def is_tails(outcome): return outcome == 'Tails'

In [5]:
# Defining an event-detection function
def get_matching_event(event_condition, sample_space):
    return set([outcome for outcome in sample_space if event_condition(outcome)])

event_conditions = [is_heads_or_tails, is_heads, is_tails, is_neither]

for event_condition in event_conditions:
    print(f"Event Condition: {event_condition.__name__}")
    event = get_matching_event(event_condition, sample_space)
    print(f'Event: {event}\n')

Event Condition: is_heads_or_tails
Event: {'Tails', 'Heads'}

Event Condition: is_heads
Event: {'Heads'}

Event Condition: is_tails
Event: {'Tails'}

Event Condition: is_neither
Event: set()



In [6]:
# Computing event probabilities
def compute_probability(event_condition, generic_sample_space):
    event = get_matching_event(event_condition, generic_sample_space)
    return len(event) / len(generic_sample_space)

for event_condition in event_conditions:
    prob = compute_probability(event_condition, sample_space)
    name = event_condition.__name__
    print(f"Probability of event arising from '{name}' is {prob}")

Probability of event arising from 'is_heads_or_tails' is 1.0
Probability of event arising from 'is_heads' is 0.5
Probability of event arising from 'is_tails' is 0.5
Probability of event arising from 'is_neither' is 0.0


In [7]:
# What would happen if that coin was biased?
# Analyzing a biased coin
weighted_sample_space = {'Heads': 4, 'Tails': 1}

# Checking the weighted sample space size
sample_space_size = sum(weighted_sample_space.values())

# Checking the weighted event size
event = get_matching_event(is_heads_or_tails, weighted_sample_space)
event_size = sum(weighted_sample_space[outcome] for outcome in event)
assert event_size == 5



# Defining a generalized event probability function
def compute_event_probability(event_condition, generic_sample_space):
    event = get_matching_event(event_condition, generic_sample_space)

    # if is not biased
    if type(generic_sample_space) == type(set()):
        return len(event) / len(generic_sample_space)
    
    # if is biased(weighted sample)
    event_size = sum(generic_sample_space[outcome] for outcome in event)
    return event_size / sum(generic_sample_space.values())



for event_condition in event_conditions:
    prob = compute_event_probability(event_condition, weighted_sample_space)
    name = event_condition.__name__
    print(f"Probability of event arising from '{name}' is {prob}")

Probability of event arising from 'is_heads_or_tails' is 1.0
Probability of event arising from 'is_heads' is 0.8
Probability of event arising from 'is_tails' is 0.2
Probability of event arising from 'is_neither' is 0.0


**Computing nontrivial probabilities**  
We’ll now solve several example problems using compute_event_probability.

In [18]:
# Problem 1: Analyzing a family with four children
# Suppose a family has four children. What is the probability that exactly two of the children are boys?
from itertools import product

possible_children = ['Boy', 'Girl']
all_combinations = set(product(possible_children, repeat=4))

def has_two_boys(outcome): return len([child for child in outcome if child == 'Boy']) == 2

prob = compute_event_probability(event_condition= has_two_boys, generic_sample_space= all_combinations)
print(f"Probability of 2 boys is {prob}")

Probability of 2 boys is 0.375


In [21]:
# Problem 2: Analyzing multiple die rolls
# Suppose we’re shown a fair six-sided die whose faces are numbered from 1 to 6. The die is rolled six times. What is the probability that these six die rolls add up to 21?
from itertools import product

possible_rolls = [1,2,3,4,5,6]
die_all_combinations = set(product(possible_rolls, repeat=6))

prob = compute_event_probability(lambda x: sum(x) == 21, die_all_combinations)
print(f"6 rolls sum to 21 with a probability of {prob}")

6 rolls sum to 21 with a probability of 0.09284979423868313


In [70]:
# Problem 3: Computing die-roll probabilities using weighted sample spaces
# we must count the number of times each sum appears across all possible die-roll combinations. 
from collections import defaultdict

weighted_sample_space = defaultdict(int)
for outcome in die_all_combinations:
    total = sum(outcome)
    weighted_sample_space[total] += 1

num_combinations = weighted_sample_space[21]
print(f"There are {num_combinations } ways for 6 die rolls to sum to 21")

# Computing the weighted event probability of die rolls
prob = compute_event_probability(lambda x: x == 21, weighted_sample_space)
print(f"6 rolls sum to 21 with a probability of {prob} \n")

# What is the benefit of using a weighted sample space over an unweighted one? Less memory usage!
# Comparing weighted to unweighted event space size
print('Number of Elements in Unweighted Sample Space:')
print(len(die_all_combinations))
print('Number of Elements in Weighted Sample Space:')
print(len(weighted_sample_space))

There are 4332 ways for 6 die rolls to sum to 21
6 rolls sum to 21 with a probability of 0.09284979423868313 

Number of Elements in Unweighted Sample Space:
46656
Number of Elements in Weighted Sample Space:
31


**Computing probabilities over interval ranges**  


In [73]:
def is_in_interval(number, minimum, maximum):
    return minimum <= number <= maximum

prob = compute_event_probability(lambda x: is_in_interval(x, 10, 21), weighted_sample_space)
print(f"Probability of interval is {prob}")

Probability of interval is 0.5446244855967078


In [99]:
def generate_coin_sample_space(num_flips=10):
    weighted_sample_space = defaultdict(int)
    for coin_flips in product(['Heads', 'Tails'], repeat=num_flips):
        heads_count = len([outcome for outcome in coin_flips if outcome == 'Heads'])
        weighted_sample_space[heads_count] += 1
    return weighted_sample_space

weighted_sample_space = generate_coin_sample_space()
assert weighted_sample_space[10] == 1
assert weighted_sample_space[9] == 10

# Computing an extreme head-count probability
prob = compute_event_probability(lambda x: is_in_interval(x, 8, 10), weighted_sample_space)
print(f"Probability of observing more than 7 heads is {prob}")

# Computing an extreme interval probability
prob = compute_event_probability(lambda x: not is_in_interval(x, 3, 7), weighted_sample_space)
print(f"Probability of observing more than 7 heads or 7 tails is {prob}")

# Analyzing extreme head counts for 20 fair coin flips
weighted_sample_space_20_flips = generate_coin_sample_space(num_flips=20)
prob = compute_event_probability(lambda x: not is_in_interval(x, 5, 15), weighted_sample_space_20_flips)
print(f"Probability of observing more than 15 heads or 15 tails is {prob}")

Probability of observing more than 7 heads is 0.0546875
Probability of observing more than 7 heads or 7 tails is 0.109375
Probability of observing more than 15 heads or 15 tails is 0.01181793212890625
